# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# read data into DF
homepage_actions = pd.read_csv('homepage_actions.csv')
# inspect first five rows
homepage_actions.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [40]:
homepage_actions['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [41]:
4264 + 3924 == len(homepage_actions)

True

In [42]:
homepage_actions['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [43]:
6328 + 1860 == len(homepage_actions)

True

In [3]:
homepage_actions.id.describe()

count      8188.000000
mean     564699.749878
std      219085.845672
min      182988.000000
25%      373637.500000
50%      566840.500000
75%      758078.000000
max      937217.000000
Name: id, dtype: float64

In [4]:
len(homepage_actions)

8188

In [44]:
homepage_actions.id.value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [45]:
homepage_actions.loc[ homepage_actions['id'] == 363314 ]

,timestamp,id,group,action
3751,2016-11-17 15:51:56.634476,363314,control,view
3752,2016-11-17 15:52:30.776850,363314,control,click


In [48]:
# Trying to understand what was done in the lab solution
clicker_ids = homepage_actions.loc[ homepage_actions['action'] == 'click', 'id'].unique()
viewer_ids = homepage_actions.loc[ homepage_actions['action'] == 'view', 'id'].unique()

In [51]:
# How many viewed?
print(f'Number of viewers: {len(viewer_ids)}')
# How many clicked?
print(f'Number of clickers: {len(clicker_ids)}')

# Now I understand why the ids series were converted into sets in the solution - to use the set minus feature
len(set(clicker_ids) - set(viewer_ids)) # should give the number of people who clicked, but didn't view

Number of viewers: 6328
Number of clickers: 1860


0

In [52]:
# conversely,
len(set(viewer_ids) - set(clicker_ids)) # should give the # of ppl who viewed the page, but didn't click

4468

In [53]:
# knowing that 4468 people viewed without clicking, yet there are 6328 viewers total tells me that 6328 - 4468 ppl
# viewed AND clicked; I think this should be equal to the number of clickers, since no clickers clicked who didn't
# also view
len(viewer_ids) - (len(set(viewer_ids) - set(clicker_ids)))

1860

In [ ]:
# so in setting up my experiment, I should grab the unique ids for those who clicked, then set minus these from those
# who viewed; that way, those who "viewed" will be converted to those who "viewed but didn't click"

In [54]:
# Next question: is there an overlap b/w experiment and control groups?
experiment_ids = homepage_actions.loc[ homepage_actions['group'] == 'experiment', 'id'].unique()
control_ids = homepage_actions.loc[ homepage_actions['group'] == 'control', 'id'].unique()
print(len(set(experiment_ids) - set(control_ids)) == len(experiment_ids))
print(len(set(control_ids) - set(experiment_ids)) == len(control_ids))

True
True


In [69]:
# I think I finally understand the solution; they created a pivot table with index "id", so each "id" corresponds to
# a row instead of having multiple rows for one id; the columns are the two values in "action", click and view; they
# use a created count column to determine the value, 1, for each column (they also fill 0s for those which don't
# have a "click" row and therefore no count for click); then they only use the click column to decide, was this person
# a clicker or a viewer only?

In [70]:
experiment['count'] = 1

In [72]:
pivoted_experiment = experiment.pivot_table(index='id', columns='action', values='count')

In [75]:
pivoted_experiment.fillna(0, inplace=True)

In [76]:
pivoted_experiment.head()

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0


In [77]:
control['count'] = 1

pivoted_control = control.pivot_table(index='id', columns='action', values='count')
pivoted_control.fillna(0, inplace=True)
pivoted_control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [78]:
# Welch's test
stats.ttest_ind(pivoted_experiment['click'], pivoted_control['click'], equal_var=False)

Ttest_indResult(statistic=2.615440020788211, pvalue=0.008932805628674201)

In [82]:
# Chi squared
obs_clicks = sum(pivoted_experiment['click'])
obs_views = len(pivoted_experiment['click']) - obs_clicks
expected_clicks = sum(pivoted_control['click'])
expected_views = len(pivoted_control['click']) - expected_clicks

stats.chisquare(f_obs=[obs_clicks, obs_views], f_exp=[expected_clicks, expected_views])

Power_divergenceResult(statistic=45.943834048640916, pvalue=1.2169234221091101e-11)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [87]:
control_rate = pivoted_control['click'].mean()
expected_clicks = control_rate * len(pivoted_experiment)
expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [91]:
# variance is np(1 - p) where n is the number of trials, and p is the expected probability of a click (based on the
# control group)
var = len(pivoted_experiment)*control_rate*(1 - control_rate)
# standard deviation is square root of variance
stddev = np.sqrt(var)
# difference b/w expected & actual clicks divided by standard deviation gives z score relative to N(0, 1)
z_score = abs(expected_clicks - pivoted_experiment['click'].sum())/stddev
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [92]:
# To calculate the p-value, I should see what the probability of obtaining this Z-score or a more extreme one is for
# a normal distribution with mean 0 and stddev 1
# I will use the survival function since I came up with a positive Z score above
stats.norm.sf(z_score,loc=0,scale=1)

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: All of my results point to rejecting the null hypothesis, and concluding that there is a statistically significant difference between the experiment and control groups.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.